In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os

In [2]:
df_train = pd.read_csv('/kaggle/input/playing-card-value/train.csv')
df_train

,Filename,Total
0,000093681.jpg,37
1,000169578.jpg,50
2,000827954.jpg,35
3,001075659.jpg,45
4,001558425.jpg,53
...,...,...
4995,998695520.jpg,42
4996,998915603.jpg,34
4997,999615042.jpg,42
4998,999668496.jpg,46


In [3]:
df_train.Total.max()

86

In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torchvision.models as models


class CustomActivation(torch.nn.Module):
    def __init__(self, scale):
        super(CustomActivation, self).__init__()
        self.scale = scale

    def forward(self, x):
        return self.scale * torch.relu(x)

    
class CNNModel(torch.nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        vgg16 = models.vgg16()
        self.features = vgg16.features
        self.conv1 = torch.nn.Conv2d(512, 16, kernel_size=4, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(32, 64)
        self.fc2 = torch.nn.Linear(64, 1) 
        self.activation = CustomActivation(100)

    def forward(self, x):
        x = self.features(x)
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.activation(self.fc2(x))
        return x
    
    
class CustomDataset(Dataset):
    def __init__(self, dataframe, directory, transform=None):
        self.dataframe = dataframe
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.directory + '/' + self.dataframe['Filename'].iloc[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.dataframe['Total'].iloc[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [5]:
from torchvision import transforms

dataset = pd.read_csv('/kaggle/input/playing-card-value/train.csv')
image_directory = '/kaggle/input/playing-card-value/train/train'

image_transforms = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
custom_dataset = CustomDataset(dataset, image_directory, transform=image_transforms)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [8]:
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

model = CNNModel()
criterion = torch.nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005)
num_epochs = 50
total_steps = len(train_loader) * num_epochs
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=warmup_steps, 
    num_training_steps=total_steps
)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:
import math

model = model.to(device)
criterion = criterion.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        scheduler.step() #Coba diilangin
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.unsqueeze(1).float())
            val_loss += loss.item() * X_batch.size(0)
    val_loss /= len(val_dataset)
    
    train_rmse = math.sqrt(train_loss)
    val_rmse = math.sqrt(val_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train RMSE: {train_rmse:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}")

Epoch [1/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [2/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [3/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [4/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [5/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [6/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [7/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [8/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [9/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [10/50], Train Loss: 2345.8700, Train RMSE: 48.4342, Val Loss: 2344.0590, Val RMSE: 48.4155
Epoch [11/50], Train Loss: 23

In [10]:
torch.save(model.state_dict(), '/kaggle/working/model_new7.pth')

In [11]:
from torchvision import transforms
from PIL import Image

image_path = "/kaggle/input/playing-card-value/test/test/003212838.jpg"
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image = Image.open(image_path).convert('RGB')
image = image_transforms(image)

output = model(image.unsqueeze(0).to(device))  
predicted_value = output.item()

prediction_data = {'Filename': image_path, 'PredictedValue': predicted_value}
prediction_data

{'Filename': '/kaggle/input/playing-card-value/test/test/003212838.jpg',
 'PredictedValue': 0.0}

In [12]:
def predict_image_reg(path):
    global model
    image_path = dirname+'/'+filename
    image_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = image_transforms(image)

    output = model(image.unsqueeze(0).to(device))  
    predicted_value = output.item()
    
    return predicted_value

In [13]:
pred_data = {
    'Filename':[],
    'Total':[]
}

for dirname, _, filenames in os.walk('/kaggle/input'):
    if dirname == '/kaggle/input/playing-card-value/test/test':
        filenames = sorted(filenames)
        for filename in filenames:

            curr_path = os.path.join(dirname, filename)
            pred_data['Filename'].append(filename)
            pred_data['Total'].append(predict_image_reg(curr_path))

In [14]:
result_df = pd.DataFrame(pred_data)
result_df[['Filename', 'Total']].to_csv('result_image_new7.csv', index=False)